In [76]:
import requests, time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

- Step 1, 2

In [77]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

trs = soup.select('tr.aos-init')
len(trs)


0

In [78]:

driver = webdriver.Chrome()
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

- Step 3

In [79]:
tr = trs[0]
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [80]:
category = tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [81]:
channel = tr.select_one('.subject > h1 > a').text.strip()
channel

'BLACKPINK'

In [82]:
subscriber = tr.select_one('.subscriber_cnt').text.strip()
view = tr.select_one('.view_cnt').text.strip()
video = tr.select_one('.video_cnt').text.strip()
subscriber, view, video

('9130만', '332억0683만', '567개')

In [83]:
# 숫자로 바꿔주는 함수
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [84]:
convert('332억0683만')

33206830000

In [85]:
data = []
for tr in trs:
    rank = tr.select_one('.rank').get_text().strip()
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    data.append([rank, category, channel, subscriber, view, video])

df = pd.DataFrame(data, columns=['rank','category','channel','subscriber','view','video'])
df.head()

,rank,category,channel,subscriber,view,video
0,1,음악/댄스/가수,BLACKPINK,91300000,33206830000,567
1,2,음악/댄스/가수,BANGTANTV,76400000,21348920000,2330
2,3,음악/댄스/가수,HYBE LABELS,72700000,29979900000,1428
3,4,음악/댄스/가수,SMTOWN,32100000,27856680000,4227
4,5,음악/댄스/가수,JYP Entertainment,27800000,20510660000,1833


- Step 6

In [86]:
data = []
for page in range(1, 11):
    url = f'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page={page}'
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('tr.aos-init')

    for tr in trs:
        rank = tr.select_one('.rank').get_text().strip()
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        data.append([rank, category, channel, subscriber, view, video])
driver.close()

df = pd.DataFrame(data, columns=['rank','category','channel','subscriber','view','video'])
df.tail()


,rank,category,channel,subscriber,view,video
995,997,미분류,성창경TV,770000,1196450000,18314
996,996,미분류,빅페이스 BIGFACE,770000,183120000,142
997,998,키즈/어린이,꾸삐KUPI,770000,570580000,986
998,1000,키즈/어린이,서은이야기 [V-log],770000,298990000,628
999,999,TV/방송,LDF TV by lottedutyfree,770000,656800000,1517


- Step 7

In [87]:
df = pd.DataFrame(data, columns=['rank','category','channel','subscriber','view','video'])
df.head()

,rank,category,channel,subscriber,view,video
0,1,음악/댄스/가수,BLACKPINK,91300000,33206830000,567
1,2,음악/댄스/가수,BANGTANTV,76400000,21348920000,2330
2,3,음악/댄스/가수,HYBE LABELS,72700000,29979900000,1428
3,4,음악/댄스/가수,SMTOWN,32100000,27856680000,4227
4,5,음악/댄스/가수,JYP Entertainment,27800000,20510660000,1833


In [90]:
df.to_csv('data/유튜브랭킹_20230919.csv', index=False)
pd.read_csv('data/유튜브랭킹_20230919.csv').tail()

,rank,category,channel,subscriber,view,video
0,997,미분류,성창경TV,770000,1196450000,18314
1,996,미분류,빅페이스 BIGFACE,770000,183120000,142
2,998,키즈/어린이,꾸삐KUPI,770000,570580000,986
3,1000,키즈/어린이,서은이야기 [V-log],770000,298990000,628
4,999,TV/방송,LDF TV by lottedutyfree,770000,656800000,1517
